<img src="logo.png">

<img src="logo2.png">

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import train_test_split

# --------------------------------- NN ---------------------------
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

sns.set()

In [2]:
df = pd.read_csv("iris_lkh.csv")

In [3]:
print(df.shape)
df.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### NN 모델 구축 시 반드 시 확인해야 할 사항

##### 모델 입력갯수 == 피쳐갯수 
* input_dim=4*  == df.shape (500, 4*)
* model.add(Dense(units=16, activation='relu',  input_dim=4  )) 


#### 모델 출력 갯수
*  중요중요중요 :: 모델 마지막 unit 수와 일치해야 한다.
* model.add(Dense(units=1,  activation='sigmoid'))
* model.add(Dense(units=3,  activation='softmax'))

In [4]:
print(df['target'].value_counts())
print(df['target'].unique())

0    50
1    50
2    50
Name: target, dtype: int64
[0 1 2]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   target        150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


## NN

### 랜덤시드 고정

In [6]:
np.random.seed(1234)
tf.random.set_seed(1234)

### X, y (학습피쳐, 타겟) 분리 

In [7]:
y = df['target']
X = df.drop(['target'], axis=1)

### 모델

*        binary_crossentropy      : 2종==sigmoid    0, 1
* sparse_categorical_crossentropy : 3종>= somtmax   0, 1, 2
*         categorical_crossentropy : 3종>= somtmax   [[1 0 0] [0 1 0]..]


In [8]:
# model = DecesionTree()  ------  M.L XXXXX
#----------------------------------------
model = Sequential()
model.add(Dense(units=16, activation='relu',  input_dim=4  ))  #input_dim == shape피쳐갯수  (506, 9)
model.add(Dense(units=8,  activation='relu'))
model.add(Dense(units=8,  activation='relu'))
model.add(Dense(units=3,  activation='softmax'))
                          #adam = momentom + adaboost
model.compile(loss='sparse_categorical_crossentropy',  optimizer='adam'  ,  metrics=['accuracy'])  #l.o.m
#----------------------------------------


### 8:2 데이터 나누기 +  학습(fit)
* initial_epoch = 0,
* steps_per_epoch = None,
* validation_steps = None,
* validation_batch_size = None,

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

### callback  (EarlyStopping, CheckPoint)
* EarlyStopping : 오버피팅 시 학습 중단
* CheckPoint : epoch 수행 후 점수가 좋아지면 해당 모델을 저장

In [10]:
stopping = EarlyStopping(monitor='val_loss',patience=10)

# os.path

import os

model_dir = './save_models/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    print("path error~!!!!")
else:
    print("path ok~!!!!")
    
    
model_path = "./save_models/{epoch:03d}_{val_loss:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_weights_only=True, save_best_only=True, period=1)

path ok~!!!!


### 학습 시작

In [11]:
# ------------ 나누기  == M.L
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=10, epochs=100 ,shuffle=True,
          callbacks=[stopping,checkpoint]
          )
# model.fit(X, y, validation_split=0.2, validation_data=(X_val,y_val), batch_size=10, epochs=100 )   #callbacks

Epoch 1/100
12/12 [==============================] - 1s 15ms/step - loss: 1.2962 - accuracy: 0.3417 - val_loss: 1.3128 - val_accuracy: 0.3000
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 1.1020 - accuracy: 0.3417 - val_loss: 1.1567 - val_accuracy: 0.3000
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 1.0014 - accuracy: 0.3417 - val_loss: 1.0670 - val_accuracy: 0.3000
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 0.9427 - accuracy: 0.4917 - val_loss: 1.0019 - val_accuracy: 0.5667
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 0.8960 - accuracy: 0.6917 - val_loss: 0.9450 - val_accuracy: 0.5667
Epoch 6/100
12/12 [==============================] - 0s 3ms/step - loss: 0.8484 - accuracy: 0.6917 - val_loss: 0.8884 - val_accuracy: 0.5667
Epoch 7/100
12/12 [==============================] - 0s 3ms/step - loss: 0.7971 - accuracy: 0.6917 - val_loss: 0.8302 - val_accuracy: 0.5667
Epoch 8/100
